In [2]:
%pip install fiftyone

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 2.3 MB 30.6 MB/s eta 0:00:01
     |████████████████████████████████| 79.9 MB 140 kB/s  eta 0:00:01    |███▊                            | 9.4 MB 58.2 MB/s eta 0:00:02     |██████████████▎                 | 35.8 MB 58.2 MB/s eta 0:00:01     |██████████████████████████████▊ | 76.6 MB 29.6 MB/s eta 0:00:01     |███████████████████████████████▍| 78.3 MB 29.6 MB/s eta 0:00:01
     |████████████████████████████████| 569 kB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 13.2 MB 6.5 MB/s eta 0:00:01     |████████████████████████████▎   | 11

In [2]:
%pip install fiftyone-db

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install pymongo

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import numpy as np
import multiprocessing as mp
import pymongo
import fiftyone


from loader import images as im
from itertools import repeat
from IPython.core.display import HTML

from tqdm import tqdm
tqdm.pandas()

# %pip install opencv-python-headless

/opt/conda/lib/python3.7/site-packages/fiftyone/db/bin/mongod: failed to launch (code 127): /opt/conda/lib/python3.7/site-packages/fiftyone/db/bin/mongod: error while loading shared libraries: libcurl.so.4: cannot open shared object file: No such file or directory


RuntimeError: Could not find mongod>=4.4

In [ ]:
textcaps_train = fiftyone.zoo.load_zoo_dataset("open-images-v6", split="train")

### Analyzing the train data

In [3]:
%cd

/root
